# monitor_workflow_and_report_results

Top-level Notebook for monitoring and reporting results of workflow DRS data access scale tests.

# Imports

In [ ]:
import os
import sys
import time
from datetime import datetime
from pathlib import Path

import psutil

In [ ]:
def add_scripts_to_python_search_path() -> None:
    cwd = Path(os.getcwd())
    for relative_path in ["scripts", "terra-workflow-scale-test-tools/scripts", "../scripts"]:
        p = cwd / relative_path
        if p.is_dir():
            print(f"Found scripts diretory: {p}")
            sys.path.insert(0, str(p.absolute()))
            return
    raise Exception("scripts directory not found")
add_scripts_to_python_search_path()
sys.path

In [ ]:
from workflow_info import WorkflowDAO

# Optionally Configure for Local/External-to-Terra Development

In [ ]:
# Enable external development setup
# TODO Set this to False for running in Terra Jupyter Cloud Environment.
external_development=True
if external_development:
    print(f"Initial CWD: {os.getcwd()}")  # Debugging
    %run ./terra-workflow-scale-test-tools/notebooks/external_development.ipynb
print(f"CWD: {os.getcwd()}")  # Debugging
print(f"PATH: {os.getenv('PATH')}")  # Debugging

# Manual Input/Configuration

Enter the workflow submission id, which is available from Job Manager
after the workflow is started.
For example:
```
 WF_SUBMISSION_ID="1a72b974-00c4-4316-86d5-7a7b1045f9ef"
```

In [ ]:
#TODO Uncomment this line when using as a template Notebook
# WF_SUBMISSION_ID = "<Enter workflow submission id as shown above.>"
#WF_SUBMISSION_ID="1a72b974-00c4-4316-86d5-7a7b1045f9ef"
# md5_n_by_m_scatter 100 files
WF_SUBMISSION_ID="32f74aa9-7779-4d28-aec7-641d26307beb"

Enter the start time for the workflow using UTC/GMT in this format:
```
2022/02/19 01:33:05
YYYY/MM/DD HH:MM:SS
```

For example, the current time in UTC in this format is:

In [ ]:
datetime.utcnow().strftime("%Y/%m/%d %H:%M:%S")

In [ ]:
#TODO Uncomment this line when using as a template Notebook
# WF_START_TIME = "<Enter UTC date/time in the format above.>"
WF_START_TIME = datetime.utcnow().strftime("%Y/%m/%d %H:%M:%S")

In [ ]:
WORKSPACE_NAMESPACE = os.getenv('WORKSPACE_NAMESPACE')
WORKSPACE_NAME = os.getenv('WORKSPACE_NAME')

In [ ]:
workflow_dao = WorkflowDAO(WORKSPACE_NAMESPACE, WORKSPACE_NAME, WF_SUBMISSION_ID)

workflow_info = workflow_dao.get_workflow_info()
# print(json.dumps(workflow_info, indent=4))
print(f"Workflow in process: {workflow_dao.is_in_process()}")
print(f"Submission time: {workflow_dao.get_submission_time()}")
print(f"Submission time: {workflow_dao.get_submission_time('%Y/%m/%d %H:%M:%S')}")

## Processing Steps to Run

In [ ]:
monitor_workflow=True
copy_workflow_logs=True
extract_timeseries_data=True
display_results=True

# Constants

## Data Access Rate Constants

In [ ]:
WORKSPACE_BUCKET=os.environ['WORKSPACE_BUCKET']
WORKSPACE_BUCKET

In [ ]:
WS_WF_GS_URI=f"{WORKSPACE_BUCKET}/{WF_SUBMISSION_ID}"
WS_WF_GS_URI

In [ ]:
TEST_RESULTS_DIR=Path('./test_results').resolve().as_posix()
! mkdir -p "{TEST_RESULTS_DIR}"
TEST_RESULTS_DIR

In [ ]:
WF_TEST_RESULTS_DIR=os.path.join(TEST_RESULTS_DIR, f"submission_{WF_SUBMISSION_ID}")
! mkdir -p "{WF_TEST_RESULTS_DIR}"
WF_TEST_RESULTS_DIR

In [ ]:
WF_TEST_RESULTS_WORKFLOW_LOGS_DIR=os.path.join(WF_TEST_RESULTS_DIR, "workflow-logs")
WF_TEST_RESULTS_WORKFLOW_LOGS_DIR

## Response Time Monitoring Constants

Currently supported values:  `BDC`

In [ ]:
PROJECT_TO_MONITOR="BDC"

Currently supported values: `DEV`

In [ ]:
TERRA_DEPLOYMENT_TIER="ALPHA"

In [ ]:
MONITORING_OUTPUT_DIR=os.path.join(WF_TEST_RESULTS_DIR,
                                   f"monitoring_data_{datetime.strptime(WF_START_TIME, '%Y/%m/%d %H:%M:%S').strftime('%Y%m%d_%H%M%S')}")
! mkdir -p "{MONITORING_OUTPUT_DIR}"
MONITORING_OUTPUT_DIR

## General Constants

In [ ]:
SCRIPTS=Path("./terra-workflow-scale-test-tools/scripts").resolve().as_posix()
SCRIPTS

In [ ]:
NOTEBOOKS=Path("./terra-workflow-scale-test-tools/notebooks").resolve().as_posix()
NOTEBOOKS

# Functions

In [ ]:
def start_monitoring_background_process() -> psutil.Process:
    print("Starting monitoring background process ...")
    process = psutil.Popen(["python3", f"{SCRIPTS}/monitor_response_times.py",
                            "--project", PROJECT_TO_MONITOR,
                            "--terra-deployment-tier", TERRA_DEPLOYMENT_TIER,
                            "--output-dir", MONITORING_OUTPUT_DIR])
    print(f"Started {process}")
    return process

In [ ]:
def stop_monitoring_background_process(process: psutil.Process) -> None:
    print("Stopping monitoring background process ...")
    process.terminate()
    process.wait(60)
    print("Stopped monitoring background process.")

In [ ]:
def wait_for_workflow_to_complete() -> None:
    # TODO Implement this
    pretend_duration = 60
    print(f"Pretending to wait for workflow to complete by sleeping {pretend_duration} seconds ...")
    time.sleep(pretend_duration)
    print("Pretending workflow is complete!")

# Monitor response times during workflow execution

In [ ]:
if monitor_workflow:
    monitoring_process = start_monitoring_background_process()

    wait_for_workflow_to_complete()

    stop_monitoring_background_process(monitoring_process)

# Copy workflow logs from the workspace bucket to the local filesystem

In [ ]:
if copy_workflow_logs:
    workflow_logs_path = Path(WF_TEST_RESULTS_WORKFLOW_LOGS_DIR)
    if not workflow_logs_path.exists():
        workflow_logs_path.mkdir(parents=True, exist_ok=False)
        # Copy the logs - this can take a long time (tens of minutes to hours)
        ! "{SCRIPTS}/copy_workflow_logs_to_local_fs.sh" -s "{WS_WF_GS_URI}" -d "{WF_TEST_RESULTS_WORKFLOW_LOGS_DIR}" > "{WF_TEST_RESULTS_WORKFLOW_LOGS_DIR}/copy_workflow_logs_to_local_fs.log" 2>&1
    else:
        print(f"The workflow-logs directory already exists: {WF_TEST_RESULTS_WORKFLOW_LOGS_DIR}")
        print("Skipping copy of the workflow logs.")
else:
    print("Currently configured to skip copying of workflow logs.")


In [ ]:
if extract_timeseries_data:
    ! "{SCRIPTS}/extract_drs_localization_timestamps.sh" -d "{WF_TEST_RESULTS_DIR}" # > "{WF_TEST_RESULTS_WORKFLOW_LOGS_DIR}/extract_drs_localization_timestamps.log" 2>&1

In [ ]:
if extract_timeseries_data:
    ! "{SCRIPTS}/extract_drs_localization_fallback_timestamps.sh" -d "{WF_TEST_RESULTS_DIR}"  # > "{WF_TEST_RESULTS_WORKFLOW_LOGS_DIR}/extract_drs_localization_fallback_timestamps.log" 2>&1

# Display the results of the workflow run

## Workflow DRS localization rates

In [ ]:
if display_results:
    %run "{NOTEBOOKS}/graph_drs_data_access_rates.ipynb"

## Service/endpoint response times

In [ ]:
if display_results:
    %run "{NOTEBOOKS}/graph_response_time_data.ipynb"